In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from scipy.sparse import hstack


In [23]:
df = pd.read_csv("data\cleaned_output_file.csv")
df2 = pd.read_csv("data\goodreads_nonfiction_books.csv")

In [28]:
#min max scaler for rating and year
scaler = MinMaxScaler()
df['Rating Scale'] = scaler.fit_transform(df[['Average Rating']])
df['Year Scale'] = scaler.fit_transform(df[['Publish Date']])



#tf idf for synop + reviews
vectorizer = TfidfVectorizer(ngram_range=(1,2))

columnstoVectorize = ("Proccessed Synopsis" , "Proccessed Review")

reviews_tfidf = vectorizer.fit_transform(df['Proccessed Review'])
synopsis_tfidf = vectorizer.fit_transform(df['Proccessed Synopsis'])

#genres, book titles authors
if df['Genres'] is str:
    df['Genres'] = df['Genres'].str.split("'")
    print(df['Genres'])
encoder = OneHotEncoder(handle_unknown='ignore')
encoded_genres = encoder.fit_transform(df['Genres'].apply(lambda x: pd.Series(x)).stack().reset_index(level=1, drop=True).to_frame('Genres'))
encoded_titles = encoder.fit_transform(df[['Book Title']])
encoded_authors = encoder.fit_transform(df[['Author']])
encoded_features = hstack([encoded_titles, encoded_authors, encoded_genres])

#combing everythng together
combined_features = hstack([encoded_features, df[['Rating Scale', 'Year Scale']], reviews_tfidf, synopsis_tfidf])
with open("text.txt", "w") as f:
    for i in range(combined_features.shape[0]):
        row = combined_features.getrow(i)
        for j in zip(row.indices, row.data):
            f.write(f"Row {i}, Column {j[0]}, Value {j[1]}\n")

In [4]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
import numpy as np

similarity_matrix = cosine_similarity(combined_features)



In [5]:
def get_recommendations(title, similarity_matrix, df, top_n=10):
    idx = df.index.get_loc(title)
    sim_scores = list(enumerate(similarity_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]
    book_indices = [i[0] for i in sim_scores]
    
    return df.iloc[book_indices].index.tolist()




In [21]:
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import sv_ttk

if 1 in df.index: 
    df.set_index('Book Title', inplace=True)
if 1 in df.index:
    df.set_index('Book Title', inplace=True)


class AutocompleteCombobox(ttk.Combobox):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.config(font=("Times New Roman", 10))
        self.book_titles = sorted(kwargs.get('values', []))  # List of book titles
        self._completion_list = self.book_titles
        self.bind('<KeyRelease>', self.handle_keyrelease)
        self._hits = []

    def handle_keyrelease(self, event):
        if event.keysym == "BackSpace":
            self.delete(self.index(tk.INSERT), tk.END)
        elif event.keysym in ["Left", "Right"]:
            return
        elif event.keysym == "Return":  # Handle Enter key
            self._update_autocomplete()
            self.event_generate('<Down>')
        else:
            self._update_autocomplete()

    def _update_autocomplete(self):
        typed = self.get()
        if typed == '':
            self._hits = self._completion_list
        else:
            self._hits = [title for title in self._completion_list if typed.lower() in title.lower()]
        if self._hits:
            self['values'] = self._hits
            self.event_generate('<Down>')

def show_synopsis(title): #box for showing synopsis, df2 to avoid lemmatized text
    synopsis = df2.loc[title]["Synopsis"]
    messagebox.showinfo(title="Synopsis", message=synopsis)

def update_recommendations(event):
    selected_book = book_var.get()
    recommendations = get_recommendations(selected_book, similarity_matrix, df)
    for i in recommendation_tree.get_children():
        recommendation_tree.delete(i)
    for book in recommendations:
        recommendation_tree.insert("", "end", text=book)

# Tkinter window setup
root = tk.Tk()
root.title("Book Recommendation System")

book_var = tk.StringVar()
book_titles = df.index.tolist()

book_dropdown = AutocompleteCombobox(root, textvariable=book_var, values=book_titles)
book_dropdown.grid(row=0, column=0, padx=10, pady=10)
book_dropdown.bind('<<ComboboxSelected>>', update_recommendations)

# Set up treeview, configuring the size
recommendation_tree = ttk.Treeview(root, height=10)
recommendation_tree.grid(row=1, column=0, padx=10, pady=10, sticky='ew')  
root.grid_columnconfigure(0, weight=1)  
recommendation_tree['columns'] = ()  
recommendation_tree.heading('#0', text='Book Title')
recommendation_tree.column('#0', stretch=tk.YES, width=400)  

# Bdouble click to show synopsis
def on_double_click(event):
    item_id = event.widget.selection()[0]
    item_text = event.widget.item(item_id, 'text')
    show_synopsis(item_text)

recommendation_tree.bind("<Double-1>", on_double_click)
sv_ttk.set_theme("dark")
root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\ieyua\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\ieyua\AppData\Local\Temp\ipykernel_19512\661104892.py", line 77, in on_double_click
    show_synopsis(item_text)
  File "C:\Users\ieyua\AppData\Local\Temp\ipykernel_19512\661104892.py", line 43, in show_synopsis
    synopsis = df2.loc[title]["Synopsis"]
               ~~~~~~~^^^^^^^
  File "c:\Users\ieyua\anaconda3\Lib\site-packages\pandas\core\indexing.py", line 1103, in __getitem__
    return self._getitem_axis(maybe_callable, axis=axis)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ieyua\anaconda3\Lib\site-packages\pandas\core\indexing.py", line 1343, in _getitem_axis
    return self._get_label(key, axis=axis)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ieyua\anaconda3\Lib\site-packages\pandas\core\indexing.py", line 1293, 